In [18]:
!python -V

Python 3.9.21


In [19]:
import pandas as pd

In [20]:
import pickle

In [21]:
import seaborn as sns
import matplotlib.pyplot as plt

In [22]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".
Download the data for January and February 2023.
Read the data for January. How many columns are there?

In [23]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

In [24]:
len(df.columns)

19

Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.
What's the standard deviation of the trips duration in January?

In [ ]:

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime # calculate duration
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60) # convert to minutes

In [26]:
df.duration.std() # Get the standard deviation of duration

np.float64(42.59435124195458)

Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).
What fraction of the records left after you dropped the outliers?

In [27]:
ol = df[(df.duration >= 1) & (df.duration <= 60)].copy() # filter outliers

In [28]:
len(ol) / len(df) * 100 # Get fraction of the records left

98.1220282212598

Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.
Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [30]:
df = ol.copy() # Remove outliers

In [ ]:
categorical = ['PULocationID', 'DOLocationID'] # Categorical features
df[categorical] = df[categorical].astype(str) # Add new column as string type
train_dicts = df[categorical].to_dict(orient='records') # Convert to dicts

dv = DictVectorizer() # Create DictVectorizer
X_train = dv.fit_transform(train_dicts) # Fit the DictVectorizer to the training data

In [35]:
X_train.shape # Count the number of features in the training data

(3009173, 515)

Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.
Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [42]:
target = 'duration'
y_train = df[target].values # Get the target variable

In [43]:
lr = LinearRegression() # Create Linear Regression model
lr.fit(X_train, y_train) # Fit the model to the training data

y_pred = lr.predict(X_train) # Make predictions on the training data

root_mean_squared_error(y_train, y_pred) # Calculate RMSE on the training data

7.649261934377987

Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).
What's the RMSE on validation?

In [46]:
def read_data(filename): # Function to read data
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime # calculate duration
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60) # convert to minutes

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy() # Remove outliers

    df[categorical] = df[categorical].astype('str') # Convert to string
    
    return df

In [47]:
df_val = read_data('./data/yellow_tripdata_2023-02.parquet') # Read validation data
val_dicts = df_val[categorical].to_dict(orient='records') # Convert to dicts

X_val = dv.transform(val_dicts) # Transform validation data
y_val = df_val.duration.values # Get the target variable

y_pred = lr.predict(X_val) # Make predictions on the validation data

In [49]:
root_mean_squared_error(y_val, y_pred) # Calculate RMSE on the validation data

7.8118183996272